## COCO division

In [1]:
#GLOBAL VARIABLES

path_train_images = "data/train2014/"
path_train_labels = "data/coco/labels/train2014/"
path_val_images = "data/val2014/"
path_val_labels = "data/coco/labels/val2014/"

MAX_file_name_train = 581921
MIN_file_name_train = 9
MAX_file_name_val = 581929
MIN_file_name_val = 42
nb_file_train = 82783
nb_file_val = 40504

nb_labels=80

nb_train = 1800
nb_val = 200


In [2]:
#UTILS

import os.path
import numpy as np
import os
import datetime
import shutil

def get_fname(file_nb):
    return (12-len(str(file_nb)))*'0'+str(file_nb)

def pair_exist(file_nb,val=False):    #val=False : check on training set | val=True : check on val set
    exist=False
    if(not val):
        t1 = os.path.exists(path_train_images+"COCO_train2014_"+get_fname(file_nb)+".jpg")
        t2 = os.path.exists(path_train_labels+"COCO_train2014_"+get_fname(file_nb)+".txt")
        if(t1 and t2):
            exist=True
    else:
        t1 = os.path.exists(path_val_images+"COCO_val2014_"+get_fname(file_nb)+".jpg")
        t2 = os.path.exists(path_val_labels+"COCO_val2014_"+get_fname(file_nb)+".txt")
        if(t1 and t2):
            exist=True
    return exist

def print_labels(nbl,percl):
    print("Label  |  Number  |  Percentage")
    print("_______________________________")
    for i in range(len(nbl)):
        print(" "+(2-len(str(i)))*'0'+str(i)+"    |  "+str(nbl[i])+(6-len(str(nbl[i])))*' '+"  |  "+str(format(percl[i]*100,'0.3f'))+"%" )
    total=np.sum(nbl)
    totalp=np.sum(percl)
    print("Total  |  "+str(total)+(8-len(str(total)))*' '+"|  "+str(totalp*100)+"%")
    

def get_all_labels(val=False):
    if (not val):
        path=path_train_labels
        mini=MIN_file_name_train
        maxi=MAX_file_name_train
        name="COCO_train2014_"
    else:
        path=path_val_labels
        mini=MIN_file_name_val
        maxi=MAX_file_name_val
        name="COCO_val2014_"
    total=0
    nbtotfile=0
    all_labels_number=np.zeros((nb_labels),dtype='int32') #use int16 if not many labels
    for file_nb in range(mini,maxi+1):
        if(pair_exist(file_nb,val=val)):
            f=open(path+name+get_fname(file_nb)+".txt","r")
            Lines=f.readlines()
            f.close()
            for line in Lines:
                all_labels_number[int(line[0:len(str(nb_labels))])]+=1
                total+=1
            nbtotfile+=1
    all_labels_percentage=np.copy(all_labels_number)
    all_labels_percentage=all_labels_percentage.astype('float64')
    all_labels_percentage/=total
    
    
    return all_labels_number, all_labels_percentage, nbtotfile


def create_dir():
    try:
        os.mkdir("data_divided")
    except:
        print("data_divided already exist")
    try:
        os.mkdir("data_divided/images")
    except:
        print("data_divided/images already exist")
    try:
        os.mkdir("data_divided/labels")
    except:
        print("data_divided/labels already exist")
    try:
        os.mkdir("data_divided/images/train2014")
    except:
        print("data_divided/images/train2014 already exist")
    try:
        os.mkdir("data_divided/images/val2014")
    except:
        print("data_divided/images/val2014 already exist")
    try:
        os.mkdir("data_divided/labels/train2014")
    except:
        print("data_divided/labels/train2014 already exist")
    try:
        os.mkdir("data_divided/labels/val2014")
    except:
        print("data_divided/labels/val2014 already exist")
    
    #create logs
    f=open("data_divided/logs","w")
    currentDT = datetime.datetime.now()
    f.write("logs version from "+str(currentDT)+"\n")
    f.close()
    

def divide_dataset(nbl,percl,val=False):
    if (not val):
        path=path_train_labels
        nbtot=nb_train
        nbtotfile=nb_file_train
        mini=MIN_file_name_train
        maxi=MAX_file_name_train
        name="COCO_train2014_"
    else:
        path=path_val_labels
        nbtot=nb_val
        nbtotfile=nb_file_val
        mini=MIN_file_name_val
        maxi=MAX_file_name_val
        name="COCO_val2014_"
        
    #optimal_nb=percl*float(nbtot)
    optimal_nb=nbl.astype('float64')*float(nbtot)/float(nbtotfile)
    actual_nb=np.zeros((nb_labels))
    filenb_chosen=[]
    file_nb=mini
    while(len(filenb_chosen)<nbtot):
                

        if (pair_exist(file_nb,val=val) and (file_nb not in filenb_chosen) ):
            diff=optimal_nb-actual_nb
            label_int=np.max(diff)
            piv=0
            for i in range(0,len(actual_nb)):
                if(diff[i]==label_int):
                    piv=i
            label_int=piv
        
            f=open(path+name+get_fname(file_nb)+".txt","r")
            Lines=f.readlines()
            f.close()
            file_label=[]
            for line in Lines:
                file_label.append(int(line[0:len(str(nb_labels))]))
                
            if (label_int in file_label):
                for i in file_label:
                    actual_nb[i]+=1
                filenb_chosen.append(file_nb)
        
        file_nb+=1
        if (file_nb>maxi):
            file_nb=mini
    
    return filenb_chosen, optimal_nb, actual_nb
          

def create_divide_dataset(filechosen,val=False):
    if (not val):
        pathl=path_train_labels
        pathi=path_train_images
        pathcopy="train2014/"
        mini=MIN_file_name_train
        maxi=MAX_file_name_train
        name="COCO_train2014_"
        f=open("data_divided/logs","a")
        f.write("\n\ntrain set - "+str(nb_train)+" files chosen :\n\n")
        f.close()
    else:
        pathl=path_val_labels
        pathi=path_val_images
        pathcopy="val2014/"
        mini=MIN_file_name_val
        maxi=MAX_file_name_val
        name="COCO_val2014_"
        f=open("data_divided/logs","a")
        f.write("\n\nval set - "+str(nb_val)+" files chosen :\n\n")
        f.close()
    
    for file_nb in range(mini,maxi+1):
        if (pair_exist(file_nb,val=val)):
            if(file_nb in filechosen):
                shutil.copy(pathi+name+get_fname(file_nb)+".jpg","data_divided/images/"+pathcopy+name+get_fname(file_nb)+".jpg")
                shutil.copy(pathl+name+get_fname(file_nb)+".txt","data_divided/labels/"+pathcopy+name+get_fname(file_nb)+".txt")
                cho="O"
            else:
                cho="X"
            
            #fill logs
            fr=open(pathl+name+get_fname(file_nb)+".txt","r")
            Lines=fr.readlines()
            fr.close()
            file_label=[]
            for line in Lines:
                file_label.append(int(line[0:len(str(nb_labels))]))
            f=open("data_divided/logs","a")
            f.write(cho+" - "+name+get_fname(file_nb)+"  Labels : ")
            
            for i in range(len(file_label)-1):
                f.write(str(file_label[i])+", ")
            f.write(str(file_label[len(file_label)-1])+"\n")
            f.close()
                
        

                
            
    

# Processing

# TRAIN

In [3]:
%%time
#get full training labels

all_labels_number,all_labels_percentage,nbtotfile=get_all_labels()


Wall time: 14min 23s


In [4]:
#update nb of file
nb_file_train=nbtotfile
print(nb_file_train)

82081


In [5]:
#print full training labels
print_labels(all_labels_number,all_labels_percentage)

Label  |  Number  |  Percentage
_______________________________
 00    |  181681  |  30.388%
 01    |  4912    |  0.822%
 02    |  30551   |  5.110%
 03    |  5972    |  0.999%
 04    |  3828    |  0.640%
 05    |  4321    |  0.723%
 06    |  3158    |  0.528%
 07    |  7047    |  1.179%
 08    |  7457    |  1.247%
 09    |  9125    |  1.526%
 10    |  1316    |  0.220%
 11    |  1372    |  0.229%
 12    |  833     |  0.139%
 13    |  6741    |  1.128%
 14    |  7117    |  1.190%
 15    |  3299    |  0.552%
 16    |  3767    |  0.630%
 17    |  4652    |  0.778%
 18    |  6458    |  1.080%
 19    |  5598    |  0.936%
 20    |  3880    |  0.649%
 21    |  903     |  0.151%
 22    |  3658    |  0.612%
 23    |  3593    |  0.601%
 24    |  6194    |  1.036%
 25    |  7753    |  1.297%
 26    |  8769    |  1.467%
 27    |  4466    |  0.747%
 28    |  4192    |  0.701%
 29    |  1861    |  0.311%
 30    |  4684    |  0.783%
 31    |  1957    |  0.327%
 32    |  4361    |  0.729%
 33    |  6

In [6]:
#create directory for new dataset
create_dir()

In [7]:
%%time
#for train dataset : chose file number for balance reduced dataset

divide_file_number_train, divide_optimal_labels_train, divide_chosen_labels_train = divide_dataset(all_labels_number,all_labels_percentage)

Wall time: 1min 29s


In [8]:
tot_op=np.sum(divide_optimal_labels_train)
tot_cho=np.sum(divide_chosen_labels_train)
opti_perc=divide_optimal_labels_train/tot_op
cho_perc=np.copy(divide_chosen_labels_train)
cho_perc=cho_perc.astype("float64")
cho_perc/=tot_cho


In [9]:
print("optimal")
print_labels(divide_optimal_labels_train,opti_perc)

optimal
Label  |  Number  |  Percentage
_______________________________
 00    |  3984.1839158879643  |  30.388%
 01    |  107.71798589198475  |  0.822%
 02    |  669.9699077740281  |  5.110%
 03    |  130.96331672372412  |  0.999%
 04    |  83.94634568292297  |  0.640%
 05    |  94.75761747542062  |  0.723%
 06    |  69.2535422326726  |  0.528%
 07    |  154.53759091629001  |  1.179%
 08    |  163.5287094455477  |  1.247%
 09    |  200.10721116945456  |  1.526%
 10    |  28.859297523178324  |  0.220%
 11    |  30.087352736930594  |  0.229%
 12    |  18.267321304565  |  0.139%
 13    |  147.8271463554294  |  1.128%
 14    |  156.07265993348034  |  1.190%
 15    |  72.34560982444171  |  0.552%
 16    |  82.60864268222853  |  0.630%
 17    |  102.01630097099206  |  0.778%
 18    |  141.62108161450274  |  1.080%
 19    |  122.76166226045004  |  0.936%
 20    |  85.08668266712151  |  0.649%
 21    |  19.80239032175534  |  0.151%
 22    |  80.21832092688929  |  0.612%
 23    |  78.792899696

In [10]:
print("chosen")
print_labels(divide_chosen_labels_train,cho_perc)

chosen
Label  |  Number  |  Percentage
_______________________________
 00    |  5268.0  |  32.555%
 01    |  138.0   |  0.853%
 02    |  840.0   |  5.191%
 03    |  136.0   |  0.840%
 04    |  88.0    |  0.544%
 05    |  105.0   |  0.649%
 06    |  75.0    |  0.463%
 07    |  164.0   |  1.013%
 08    |  173.0   |  1.069%
 09    |  213.0   |  1.316%
 10    |  33.0    |  0.204%
 11    |  35.0    |  0.216%
 12    |  23.0    |  0.142%
 13    |  155.0   |  0.958%
 14    |  172.0   |  1.063%
 15    |  77.0    |  0.476%
 16    |  96.0    |  0.593%
 17    |  107.0   |  0.661%
 18    |  150.0   |  0.927%
 19    |  129.0   |  0.797%
 20    |  92.0    |  0.569%
 21    |  24.0    |  0.148%
 22    |  90.0    |  0.556%
 23    |  83.0    |  0.513%
 24    |  176.0   |  1.088%
 25    |  235.0   |  1.452%
 26    |  269.0   |  1.662%
 27    |  102.0   |  0.630%
 28    |  108.0   |  0.667%
 29    |  45.0    |  0.278%
 30    |  108.0   |  0.667%
 31    |  49.0    |  0.303%
 32    |  100.0   |  0.618%
 33 

In [11]:
print(len(divide_file_number_train))
print(divide_file_number_train)

1800
[36, 49, 61, 77, 86, 109, 110, 113, 127, 149, 151, 165, 201, 260, 308, 315, 322, 326, 368, 370, 382, 389, 419, 431, 436, 443, 446, 471, 510, 529, 531, 532, 542, 572, 625, 634, 659, 673, 684, 690, 716, 722, 723, 731, 735, 781, 790, 795, 821, 828, 839, 882, 897, 901, 913, 925, 927, 934, 943, 960, 965, 984, 1006, 1011, 1014, 1053, 1098, 1102, 1107, 1108, 1144, 1145, 1183, 1200, 1204, 1216, 1232, 1237, 1271, 1295, 1298, 1307, 1308, 1311, 1315, 1319, 1330, 1332, 1350, 1359, 1360, 1375, 1381, 1390, 1397, 1401, 1403, 1407, 1424, 1431, 1451, 1453, 1488, 1523, 1527, 1536, 1548, 1569, 1579, 1580, 1586, 1596, 1637, 1639, 1670, 1706, 1720, 1732, 1756, 1762, 1764, 1774, 1785, 1790, 1804, 1811, 1815, 1822, 1837, 1864, 1875, 1888, 1895, 1900, 1911, 1915, 1924, 1941, 1942, 1943, 1947, 1958, 1966, 1994, 2001, 2007, 2055, 2056, 2072, 2083, 2089, 2114, 2135, 2184, 2211, 2229, 2258, 2278, 2281, 2337, 2369, 2372, 2377, 2400, 2402, 2415, 2444, 2445, 2446, 2459, 2466, 2470, 2498, 2536, 2543, 2555, 2560,

In [12]:
%%time
#copy files chosen update logs
create_divide_dataset(divide_file_number_train)

Wall time: 7min 50s


# VAL

In [13]:
%%time
#get full val labels

all_labels_number_val,all_labels_percentage_val,nbtotfile_val=get_all_labels(val=True)


Wall time: 7min 8s


In [14]:
#update nb of file
nb_file_val=nbtotfile_val
print(nb_file_val)

40137


In [15]:
#print full val labels
print_labels(all_labels_number_val,all_labels_percentage_val)

Label  |  Number  |  Percentage
_______________________________
 00    |  86348   |  29.939%
 01    |  2458    |  0.852%
 02    |  14898   |  5.165%
 03    |  3049    |  1.057%
 04    |  1444    |  0.501%
 05    |  2023    |  0.701%
 06    |  1602    |  0.555%
 07    |  3337    |  1.157%
 08    |  3543    |  1.228%
 09    |  4351    |  1.509%
 10    |  650     |  0.225%
 11    |  686     |  0.238%
 12    |  510     |  0.177%
 13    |  3490    |  1.210%
 14    |  3852    |  1.336%
 15    |  1669    |  0.579%
 16    |  1951    |  0.676%
 17    |  2187    |  0.758%
 18    |  3119    |  1.081%
 19    |  2788    |  0.967%
 20    |  1856    |  0.644%
 21    |  462     |  0.160%
 22    |  1877    |  0.651%
 23    |  1767    |  0.613%
 24    |  2890    |  1.002%
 25    |  3919    |  1.359%
 26    |  4113    |  1.426%
 27    |  2234    |  0.775%
 28    |  2219    |  0.769%
 29    |  935     |  0.324%
 30    |  2180    |  0.756%
 31    |  793     |  0.275%
 32    |  2198    |  0.762%
 33    |  2

In [16]:
%%time
#for val dataset : chose files for balance reduced dataset

divide_file_number_val, divide_optimal_labels_val, divide_chosen_labels_val = divide_dataset(all_labels_number_val,all_labels_percentage_val,val=True)

Wall time: 5.53 s


In [17]:
tot_op_val=np.sum(divide_optimal_labels_val)
tot_cho_val=np.sum(divide_chosen_labels_val)
opti_perc_val=divide_optimal_labels_val/tot_op_val
cho_perc_val=np.copy(divide_chosen_labels_val)
cho_perc_val=cho_perc_val.astype("float64")
cho_perc_val/=tot_cho_val


In [18]:
print("optimal")
print_labels(divide_optimal_labels_val,opti_perc_val)

optimal
Label  |  Number  |  Percentage
_______________________________
 00    |  430.26633779305877  |  29.939%
 01    |  12.248050427286543  |  0.852%
 02    |  74.23574258165782  |  5.165%
 03    |  15.192964097964472  |  1.057%
 04    |  7.1953559060218755  |  0.501%
 05    |  10.080474375264718  |  0.701%
 06    |  7.982659391583826  |  0.555%
 07    |  16.628048932406507  |  1.157%
 08    |  17.654533223708796  |  1.228%
 09    |  21.680743453671177  |  1.509%
 10    |  3.2389067444004285  |  0.225%
 11    |  3.418292348705683  |  0.238%
 12    |  2.5412960609911055  |  0.177%
 13    |  17.39043775070384  |  1.210%
 14    |  19.194259660662233  |  1.336%
 15    |  8.316515932929716  |  0.579%
 16    |  9.72170316665421  |  0.676%
 17    |  10.89767546154421  |  0.758%
 18    |  15.541769439669133  |  1.081%
 19    |  13.892418466751376  |  0.967%
 20    |  9.248324488626455  |  0.644%
 21    |  2.302115255250766  |  0.160%
 22    |  9.352966091137853  |  0.651%
 23    |  8.804843

In [19]:
print("chosen")
print_labels(divide_chosen_labels_val,cho_perc_val)

chosen
Label  |  Number  |  Percentage
_______________________________
 00    |  534.0   |  29.983%
 01    |  11.0    |  0.618%
 02    |  92.0    |  5.166%
 03    |  24.0    |  1.348%
 04    |  5.0     |  0.281%
 05    |  9.0     |  0.505%
 06    |  7.0     |  0.393%
 07    |  21.0    |  1.179%
 08    |  21.0    |  1.179%
 09    |  32.0    |  1.797%
 10    |  1.0     |  0.056%
 11    |  2.0     |  0.112%
 12    |  0.0     |  0.000%
 13    |  20.0    |  1.123%
 14    |  44.0    |  2.471%
 15    |  6.0     |  0.337%
 16    |  7.0     |  0.393%
 17    |  10.0    |  0.561%
 18    |  14.0    |  0.786%
 19    |  13.0    |  0.730%
 20    |  7.0     |  0.393%
 21    |  0.0     |  0.000%
 22    |  8.0     |  0.449%
 23    |  6.0     |  0.337%
 24    |  13.0    |  0.730%
 25    |  19.0    |  1.067%
 26    |  26.0    |  1.460%
 27    |  14.0    |  0.786%
 28    |  12.0    |  0.674%
 29    |  4.0     |  0.225%
 30    |  11.0    |  0.618%
 31    |  4.0     |  0.225%
 32    |  12.0    |  0.674%
 33 

In [20]:
print(len(divide_file_number_val))
print(divide_file_number_val)

200
[74, 136, 139, 192, 241, 257, 294, 328, 338, 357, 360, 395, 397, 415, 428, 459, 474, 488, 520, 536, 544, 564, 569, 589, 623, 641, 692, 693, 730, 761, 764, 785, 810, 831, 836, 872, 885, 923, 962, 969, 974, 985, 999, 1000, 1089, 1146, 1149, 1164, 1176, 1180, 1244, 1268, 1270, 1290, 1292, 1296, 1342, 1353, 1369, 1398, 1404, 1436, 1464, 1490, 1554, 1563, 1584, 1591, 1592, 1599, 1626, 1655, 1667, 1668, 1700, 1722, 1757, 1761, 1799, 1841, 1856, 1948, 1955, 1960, 1987, 2006, 2014, 2124, 2142, 2153, 2154, 2171, 2179, 2191, 2225, 2235, 2239, 2261, 2529, 2532, 2562, 2640, 2822, 2890, 2985, 3145, 3337, 3590, 3595, 3711, 3817, 3849, 3934, 4108, 4916, 5595, 5802, 6608, 6701, 7214, 7297, 8583, 8718, 8762, 8922, 9186, 9274, 9527, 9590, 10785, 11172, 11198, 12062, 12153, 14733, 15278, 18928, 19358, 19723, 19904, 20381, 20395, 20608, 20837, 21614, 22870, 25316, 25595, 26209, 26432, 26445, 26564, 28463, 29253, 29484, 30213, 31596, 32056, 32082, 32597, 32817, 34212, 35474, 35599, 35672, 35952, 36311,

In [21]:
%%time
#copy files chosen update logs
create_divide_dataset(divide_file_number_val, val=True)

Wall time: 1min 17s
